In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from graph_construction import LoadData

In [3]:
from edges import EdgeWeights
from Hyperbolic import HyperbolicEmbedding

In [63]:
from evaluate import Evaluate

In [5]:
config = {
    'dataset': 'marujo',
    'data_dir': 'data',
    }

marujo_data = LoadData(config)

2021-01-24 20:03:32 WARNING: Can not find mwt: default from official model list. Ignoring it.
2021-01-24 20:03:32 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |
| lemma     | ewt     |
| depparse  | ewt     |

2021-01-24 20:03:32 INFO: Use device: gpu
2021-01-24 20:03:32 INFO: Loading: tokenize
2021-01-24 20:03:34 INFO: Loading: pos
2021-01-24 20:03:35 INFO: Loading: lemma
2021-01-24 20:03:35 INFO: Loading: depparse
2021-01-24 20:03:37 INFO: Done loading processors!


In [50]:
data = marujo_data.data[1]['text']
graph = marujo_data.construct_graph(data)

In [ ]:
#Weighted Distance Matrix
word_vec_map = marujo_data.vectorization(data)
edges = EdgeWeights(graph, word_vec_map)
distance_matrix = edges.generate_distance()

In [7]:
#Adjacency distance Matrix
import networkx as nx
distance_matrix = nx.to_numpy_matrix(graph)

In [223]:
he.visualize()

In [43]:
he = HyperbolicEmbedding(distance_matrix, graph)

In [44]:
W = he.get_tree_rep()

In [45]:
new_graph = he.get_graph()

In [66]:
ev = Evaluate('data', 'marujo')

In [85]:
ev.evaluate('multipartiterank', top=10)

100%|██████████| 450/450 [00:00<00:00, 1036.41it/s]


(0.4103068783068783, 0.10421957889449626, 0.1587721592102656)